In [74]:
import numpy as np 
import torch
import torch.nn as nn  
import math
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [75]:
from datasets import load_dataset

ds = load_dataset("Teejeigh/raw_friends_series_transcript")

In [76]:
df = ds['train']['text']
# next char is easier

In [77]:
concatenated = "\n".join(ds['train']['text'])
chars = sorted((set(concatenated)))
print(chars)
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [78]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # take a string output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # take a list of integers output a string

print(encode("hii there"))
print(decode(encode("hii there")))

# For more complex tokenizers see "tiktoken"

In [79]:
import torch 

data = torch.tensor(encode(concatenated), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [80]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # For example, BERT tokenizer
# tokenized_texts = tokenizer(df, padding=True, truncation=True, return_tensors="pt")# 

In [81]:
n = int(0.9 * len(data))
train = data[:n]
val = data[n:]

In [82]:
block_size = 8 # chunk

x = train[:block_size]
y = train[1:block_size+1]
for t in range(len(x) - 1):
    print(x[:t + 1])
    print(y[t])

In [83]:
import random 

torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # gen small batch of data
    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, )) # random offsets (starting indexes)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")

In [89]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # C is the vocab size in this case = emb dim
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=20)[0].tolist()))

In [91]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [92]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

In [48]:
# consider the following toy example: version 1
# We get the mean of the previous, for current T, we want mean untill that T (including T). [1 2 3], if we are at 2, we have att mean 2+1/2 = 1.5 :)
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [56]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

In [93]:
wei

In [65]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

In [136]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
# input is (B, T) - B - batch size, T - sequence length
# input vectors = [23 23 21 41 32] - 1
#           [2 23 203 93 55] - 2 

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 20000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# -----------
# here are all the unique characters that occur in this text
text = "\n".join(df)
chars = sorted((set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """one head of self-attention"""
    def __init__(self, head_size = 16, n_emb = n_embd):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.queries = nn.Linear(n_emb, head_size, bias = False)
        self.keys = nn.Linear(n_emb, head_size, bias = False)
        self.values = nn.Linear(n_emb, head_size, bias = False) 
        self.dropout = nn.Dropout(0.1)
        self.embedding_dim = head_size

    def forward(self, x):
        B, T, C = x.shape

        q = self.queries(x) # B T 16
        k = self.keys(x) # B T 16 we want -> B 16 T 
        mask = (q @ k.transpose(-2, -1)) / math.sqrt(self.embedding_dim) # 1. dot product K * V
        tril = torch.tril(torch.ones(T, T))
        mask = mask.masked_fill(tril == 0, float('-inf'))
        mask = F.softmax(mask, dim=-1)
        # add dropout later 

        v = self.values(x)
        out = mask @ v
        return out

class MultiHead(nn.Module):
    """ multiple heads of self-attention """
    def __init__(self, num_heads, head_size, n_emb = n_embd):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_emb, n_emb) # ?
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

class MLP(nn.Module):
    def __init__(self, n_embd = n_embd):
        super().__init__()
        self.layer1 = nn.Linear(n_embd, n_embd * 2) # try * 4
        self.layer2 = nn.Linear(2 * n_embd, n_embd)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x): # x is the output from the self attention 
        x = self.relu(self.layer1(x))
        x = self.dropout(self.layer2(x))
        return x
    
class Block(nn.Module):
    def __init__(self, n_embd, n_head) -> None:
        super().__init__()
        head_size = n_embd // n_head # 64 usually
        self.self_attn = MultiHead(n_head, head_size)
        self.mlp = MLP(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.self_attn(self.ln1(x)) # first x is residual connection
        x = x + self.mlp(self.ln2(x)) # same here (RESIDUAL CONNECTION)
        return x 

class PositionalEncoding(nn.Module):
    def __init__(self, n_embd, max_len=5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, n_embd, 2) * (-math.log(10000.0) / n_embd))
        pe = torch.zeros(max_len, 1, n_embd)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        # This is typically used to register a buffer that should not to be considered a model parameter.
    def forward(self, x):
        return self.pe[:x.size(0)] # + x do we need this + ?
    

class Decoder(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, n_embd)
        self.positional_embedding = PositionalEncoding(n_embd, block_size) 
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        #self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, x, targets = None):
        input_emb = self.embedding(x)
        pos_emb = self.positional_embedding(input_emb)
        x = input_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens - ?
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

model = Decoder()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device = device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))# logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=20)[0].tolist()))